<img src="https://raw.githubusercontent.com/OGGM/oggm/master/docs/_static/logo.png" width="40%"  align="left">

In [ ]:
from __init__ import DATA_DIR, PLOT_DIR

In [ ]:
%matplotlib inline

In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import glob, os
import salem
import matplotlib.pyplot as plt
import seaborn as sns
from oggm import utils
import cartopy
import cartopy.crs as ccrs
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
if sns.__version__ != '0.8.1':
    raise ImportError('These scripts need seaborn version 0.8.1 to work properly!')

In [ ]:
# Get the RGI
frgi = '/home/mowglie/Documents/OGGM_Experiments/rgi60_allglaciers.csv'
rgi_dir = utils.get_rgi_dir(version='6')
if not os.path.exists(frgi):
    # one time action only
    fs = list(sorted(glob.glob(rgi_dir + "/*/*_rgi60_*.shp")))[2:]
    out = []
    for f in fs:
        sh = gpd.read_file(f).set_index('RGIId')
        del sh['geometry']
        out.append(sh)
    mdf = pd.concat(out)
    mdf.to_csv(frgi)
mdf = pd.read_csv(frgi, index_col=0, converters={'Form': str, 'TermType': str, 'RGIFlag':str, 'BgnDate':str, 
                                                 'EndDate':str, 'O1Region': str, 'O2Region':str, 'Name':str})
mdf['RGI_REG'] = [rid.split('-')[1].split('.')[0] for rid in mdf.index]
# Read glacier attrs
gtkeys = {'0': 'Glacier',
          '1': 'Ice cap',
          '2': 'Perennial snowfield',
          '3': 'Seasonal snowfield',
          '9': 'Not assigned',
          }
ttkeys = {'0': 'Land-terminating',
          '1': 'Marine-terminating',
          '2': 'Lake-terminating',
          '3': 'Dry calving',
          '4': 'Regenerated',
          '5': 'Shelf-terminating',
          '9': 'Not assigned',
          }
mdf['GlacierType'] = [gtkeys[g] for g in mdf.Form]
mdf['TerminusType'] = [ttkeys[g] for g in mdf.TermType]
mdf['IsTidewater'] = [ttype in ['Marine-terminating', 'Lake-terminating'] for ttype in mdf.TerminusType]
mdf['RGIId'] = mdf.index.values

In [ ]:
df = salem.read_shapefile('/home/mowglie/disk/Data/GIS/SHAPES/RGI/RGI_V6/00_rgi60_regions/00_rgi60_O1Regions.shp')

In [ ]:
fac = 5
mdf['CenLonC'] = np.round(mdf.CenLon * fac).astype(int)
mdf['CenLatC'] = np.round(mdf.CenLat * fac).astype(int)
mdf['CenC'] = ['{}_{}'.format(lon, lat) for lon, lat in zip(mdf['CenLonC'], mdf['CenLatC'])]

In [ ]:
selids = []
for i, g in mdf.groupby('CenC'):
    dis = (g.CenLon - g.CenLonC/fac)**2 + (g.CenLat - g.CenLatC/fac)**2
    selids.append(dis.idxmin())

In [ ]:
smdf = mdf.loc[selids]
len(smdf)

In [ ]:
from oggm import utils
dfr = pd.read_csv(utils.get_demo_file('oggm_ref_tstars_rgi6_cru4.csv'), index_col=0)
dfr.columns

In [ ]:
f = plt.figure(figsize=(12, 7))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
ax.stock_img()
# ax.add_feature(cartopy.feature.COASTLINE
# ax.coastlines('50m')

ax.scatter(smdf.CenLon.values, smdf.CenLat.values, color='C3', marker='.', s=3)
ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

did1 = False
did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 2
    ty = r.min_y + 2
    fs = 12
    t = r.RGI_CODE
    if t == 1:
        tx = r.min_x + 35
        if did1:
            continue
        did1 = True
    if t == 5:
        tx = r.min_x + 19
    if t == 6:
        ty = r.min_y - 7.
    if t == 8:
        tx = r.min_x + 26
    if t == 11:
        tx = r.min_x - 8.5
    if t == 10:
        ty = r.min_y + 17
        if not did10:
            did10 = True
            continue
    if t == 13:
        ty = r.min_y + 11
        tx = r.min_x + 18
    if t == 15:
        tx = r.min_x + 21
    if t == 17:
        ty = r.min_y + 25
    if t == 18:
        ty = r.min_y + 17
        
    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs, 
            bbox=dict(facecolor='w', edgecolor='k', alpha=0.7))

plt.tight_layout();
plt.savefig(os.path.join(PLOT_DIR, 'rgi_overview_a.pdf'), bbox_inches='tight')

In [ ]:
area_per_reg = mdf[['Area', 'O1Region']].groupby('O1Region').sum()

In [ ]:
n, _ = utils.parse_rgi_meta('6')
area_per_reg['RegName'] = ['{:02d}: {} '.format(int(i), n.loc[int(i)].values[0]) for i in area_per_reg.index]
area_per_reg = area_per_reg.sort_values(by='RegName')

In [ ]:
area_per_reg['Area'] = area_per_reg['Area']
area_per_reg['Area (% of global)'] = area_per_reg['Area'] / area_per_reg.Area.sum() * 100

In [ ]:
area_per_reg['N Glaciers'] = mdf.groupby('O1Region').count().Name

In [ ]:
dfe = pd.read_csv(DATA_DIR + 'error_summary.csv', index_col=0)
dfe = dfe.groupby('rgi_region').sum()
dfe.index = ['{:02d}'.format(int(s)) for s in dfe.index]
area_per_reg['ErrorArea'] = dfe.rgi_area_km2

In [ ]:
area_per_reg

In [ ]:
area_per_reg['OGGM errors (% area)'] = area_per_reg['ErrorArea'] / area_per_reg['Area'] * 100

In [ ]:
f, (ax1, ax2, ax3) = plt.subplots(1, 3, sharey=True, figsize=(12, 5))
sns.barplot(ax=ax1, x="N Glaciers", y="RegName", data=area_per_reg, color='Grey');
sns.barplot(ax=ax2, x="Area (% of global)", y="RegName", data=area_per_reg, color='Grey');
sns.barplot(ax=ax3, x="OGGM errors (% area)", y="RegName", data=area_per_reg, color='Grey');
ax3.text(0.01, 0.01, 'NA', transform=ax3.transAxes, fontsize=14)
ax1.set_ylabel('')
ax2.set_ylabel('')
ax3.set_ylabel('')
r = ax1.set_yticklabels(area_per_reg.RegName, ha='left')
plt.draw()
yax = ax1.get_yaxis()
pad = max(T.label.get_window_extent().width for T in yax.majorTicks)
yax.set_tick_params(pad=pad)
plt.tight_layout();
plt.savefig(os.path.join(PLOT_DIR, 'rgi_overview_b.pdf'), bbox_inches='tight');